<a href="https://colab.research.google.com/github/smbsues/Capstone/blob/main/Capstone_Part_1_(Sent_Calculations).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install stanza

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import stanza
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def upload_data(file):
    with open(file, 'r') as f:
      next(f)
      lines = f.readlines()
      data = []
      for line in lines:
        line_strp = line.rstrip('\n')
        data.append(line_strp)
      return data

english = upload_data('/content/drive/MyDrive/Capstone/OpenSubtitles.de-en.en')

german_gold  = upload_data('/content/drive/MyDrive/Capstone/OpenSubtitles.de-en.de')

english = english[0:10000]

german_gold = german_gold[0:10000]

#english

In [ ]:
def check_length(len1, len2):
  if len1 == len2:
    return True
  else:
    return False

check_length(len(english), len(german_gold))

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load in tokenizer and model from huggingface documentation
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")

# Load in Stanza sentiment analysis models for English and German
nlp_en = stanza.Pipeline(lang='en', processors='tokenize,sentiment', tokenize_no_ssplit=True)
nlp_de = stanza.Pipeline(lang='de', processors='tokenize,sentiment', tokenize_no_ssplit=True)

sentiments_dict = {}

for i, (d_g, e) in enumerate(zip(german_gold, english)):
  sentiments_dict[i] = {'de_gold': d_g, 'dg_sentences': [], 'dg_sentiments': [],
                        'en': e, 'en_sentences': [], 'en_sentiments': [],
                        'de_trans': '', 'dt_sentences': [], 'dt_sentiments': []}
  # German gold (original parallel text)  
  de_g_line = nlp_de(d_g)
  en_line = nlp_en(e)
  for sentence in de_g_line.sentences:
    de_g_sent = sentence.sentiment
    sentiments_dict[i]['dg_sentences'].append(sentence)
    sentiments_dict[i]['dg_sentiments'].append(de_g_sent)
  for sentence in en_line.sentences:
    en_sent = sentence.sentiment
    sentiments_dict[i]['en_sentences'].append(sentence)
    sentiments_dict[i]['en_sentiments'].append(en_sent)
  # Translate full en text to german
  translated = model.generate(**tokenizer(e, return_tensors="pt", padding=True))
  de_translated = tokenizer.decode(translated[0], skip_special_tokens=True)
  de_t_line = nlp_de(de_translated)  # Changed this to sentiment analyze the right doc
  sentiments_dict[i]['de_trans'] = de_translated
  for sentence in de_t_line.sentences:
    de_t_sent = sentence.sentiment
    sentiments_dict[i]['dt_sentences'].append(sentence)
    sentiments_dict[i]['dt_sentiments'].append(de_t_sent)

In [ ]:
# sentiments_dict[i] = {'de_gold': d_g, 'dg_sentences': [], 'dg_sentiments': [],
#                         'en': e, 'en_sentences': [], 'en_sentiments': [],
#                         'de_trans': '', 'dt_sentences': [], 'dt_sentiments': []}

# protodf = [['de_gold', 'en', 'de_trans', 'sent_gold', 'sent_en', 'sent_trans']]
# for i, data in sentiments_dict.items():
#     protodf.append([data['de_gold'], data['en'], data['de_trans'],
#                     np.mean(data['dg_sentiments']), np.mean(data['en_sentiments']),
#                     np.mean(data['dt_sentiments'])])
# sentiments_df = pd.DataFrame(protodf[1:], columns=protodf[0])

# sentiments_df.dropna(axis=0)